In [1]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 4.1 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [3]:
import os
from pyngrok import ngrok

In [4]:
ngrok.kill()

In [5]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://juniper-unvoided-wispily.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://juniper-unvoided-wispily.ngrok-free.dev


True

In [6]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [7]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [8]:
result = stateful_query("簡介比特幣")
print(result)

比特幣（Bitcoin，縮寫：BTC 或 XBT，符號：₿）是一種基於去中心化、點對點網路，並採用區塊鏈作為底層技術的加密貨幣。它由一位化名中本聰（Satoshi Nakamoto）的匿名人士或團隊於2008年提出，並於2009年正式問世。

**比特幣的主要特點包括：**
*   **去中心化**：比特幣網路不受任何中央機構、銀行或政府控制，而是由全球分散的電腦（稱為節點或礦工）共同維護運作。
*   **區塊鏈技術**：所有比特幣交易都記錄在一個公開、共享且不可竄改的數位帳本上，這就是區塊鏈。每筆交易的有效性都需經由網路中的節點驗證並打包成區塊，再添加到區塊鏈中。
*   **總量有限**：比特幣的總供應量被設計為2100萬枚，這使得它具有稀缺性，類似於數位黃金，能夠對抗傳統貨幣的通脹問題。預計挖礦活動將持續到2140年達到上限。
*   **點對點交易**：使用者可以透過加密錢包軟體，在無需銀行或支付公司等中介機構的情況下，直接進行交易轉帳。
*   **加密安全性**：比特幣利用數位加密演算法來確保資產與交易的安全。每筆交易都透過私鑰進行簽名，以證明交易來自錢包擁有者，並防止交易被篡改。
*   **公開透明**：雖然交易雙方的身份是匿名的，但所有交易記錄細節都是公開、透明且可追溯的。

**運作方式簡述：**
使用者需透過比特幣錢包來儲存和管理比特幣。當發起一筆交易時，這筆交易會被廣播到整個比特幣網路。網路中的「礦工」會驗證這些交易的合法性，並將其打包成一個「區塊」。透過「工作量證明」（Proof of Work）機制，礦工們競爭解決一道複雜的數學難題，成功解決的礦工將有權將新區塊添加到區塊鏈上，並獲得新發行的比特幣作為獎勵以及交易手續費。這個過程確保了交易的真實性與安全性，並維護了整個網路的穩定性。

比特幣的價值除了作為一種交易媒介，也被許多人視為一種儲值工具或投資資產。


In [10]:
result2 = stateful_query("發明者是誰？")
print(result2)

比特幣的發明者是一位化名為**中本聰（Satoshi Nakamoto）**的匿名人士或團隊。

儘管中本聰的名字在全球廣為人知，與比特幣的發明和區塊鏈的開發緊密相關，但其真實身份至今仍然是個謎。 中本聰於2008年10月31日在一個加密學郵件列表上發表了題為《比特幣：一種點對點電子現金系統》（Bitcoin: A Peer-to-Peer Electronic Cash System）的白皮書，詳細闡述了比特幣的概念和底層技術。

多年來，曾有多位人士被猜測可能是中本聰，包括電腦科學家與密碼學家尼克·薩博（Nick Szabo）、密碼學先驅哈爾·芬尼（Hal Finney）以及澳大利亞企業家克雷格·賴特（Craig Wright）等，但都沒有確鑿的證據能證實他們的身份。 其中，克雷格·賴特曾於2016年公開表示自己就是中本聰，並提供了相關證據，但此說法仍存在爭議。

中本聰在2010年12月後便停止參與比特幣的開發，從此消失在公眾視野中，這也使得比特幣真正實現了去中心化的目標。


In [ ]:
from flask import Flask, request, abort
import logging
import os
import time
from google.genai import types

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    FileMessageContent
)

app = Flask(__name__)

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
app.logger.setLevel(logging.INFO)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)

# 儲存檔案的目錄
UPLOAD_DIR = "/content/uploaded_files"
os.makedirs(UPLOAD_DIR, exist_ok=True)

# 儲存每個使用者的對話 session 和上傳的檔案
user_sessions = {}  # {user_id: {"chat": chat_object, "uploaded_file": gemini_file}}

def get_user_session(user_id):
    """取得或建立使用者的對話 session"""
    if user_id not in user_sessions:
        # 建立新的對話 session
        new_chat = client.chats.create(
            model="gemini-2.5-flash",
            config=GenerateContentConfig(
                system_instruction="你是一個中文的AI助手，請用繁體中文回答。如果使用者有提供參考文件，請根據文件內容回答問題。",
                tools=[google_search_tool],
                response_modalities=["TEXT"],
            )
        )
        user_sessions[user_id] = {
            "chat": new_chat,
            "uploaded_file": None
        }
    return user_sessions[user_id]

def download_line_file(message_id, file_name):
    """從 LINE 下載使用者上傳的檔案"""
    with ApiClient(configuration) as api_client:
        line_bot_blob_api = MessagingApiBlob(api_client)
        file_content = line_bot_blob_api.get_message_content(message_id)

        file_path = os.path.join(UPLOAD_DIR, file_name)

        with open(file_path, 'wb') as f:
            f.write(file_content)

        return file_path

def upload_file_to_gemini(file_path):
    """上傳檔案到 Gemini Files API"""
    uploaded_file = client.files.upload(
        file=file_path,
        config={'display_name': os.path.basename(file_path)}
    )

    # 等待檔案處理完成
    while uploaded_file.state.name == "PROCESSING":
        print("檔案處理中...")
        time.sleep(1)
        uploaded_file = client.files.get(name=uploaded_file.name)

    if uploaded_file.state.name == "FAILED":
        raise Exception("檔案上傳處理失敗")

    return uploaded_file

def query_with_rag(user_id, question):
    """使用 RAG 模式回答問題"""
    session = get_user_session(user_id)
    uploaded_file = session["uploaded_file"]

    if uploaded_file:
        # 有上傳檔案，使用 RAG 模式
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=[
                types.Content(
                    role="user",
                    parts=[
                        types.Part.from_uri(
                            file_uri=uploaded_file.uri,
                            mime_type=uploaded_file.mime_type
                        ),
                        types.Part.from_text(text=f"請根據上述提供的檔案內容，用繁體中文回答這個問題：{question}")
                    ]
                )
            ]
        )
        return response.text
    else:
        # 沒有上傳檔案，使用一般多輪對話
        response = session["chat"].send_message(message=question)
        return response.text

@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    """處理文字訊息"""
    text = event.message.text
    user_id = event.source.user_id

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        if text.startswith('AI '):
            prompt = text[3:]
            try:
                # 使用 RAG 或一般對話
                reply_text = query_with_rag(user_id, prompt)

                # 檢查是否有上傳檔案，加上提示
                session = get_user_session(user_id)
                if session["uploaded_file"]:
                    reply_text = f"📄 [RAG 模式]\n\n{reply_text}"

                line_bot_api.reply_message_with_http_info(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=[TextMessage(text=reply_text)]
                    )
                )
            except Exception as e:
                line_bot_api.reply_message_with_http_info(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=[TextMessage(text=f"❌ 發生錯誤：{str(e)}")]
                    )
                )
        elif text == "清除文件":
            # 清除使用者上傳的檔案
            session = get_user_session(user_id)
            session["uploaded_file"] = None
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="✅ 已清除上傳的文件，恢復一般對話模式。")]
                )
            )
        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="請輸入「AI 問題」來開始對話\n或上傳 TXT/PDF 檔案啟用 RAG 模式")]
                )
            )

@handler.add(MessageEvent, message=FileMessageContent)
def handle_file_message(event):
    """處理使用者上傳的檔案"""
    user_id = event.source.user_id
    file_name = event.message.file_name

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        # 檢查檔案類型
        if not (file_name.endswith('.txt') or file_name.endswith('.pdf')):
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="⚠️ 目前只支援 TXT 或 PDF 檔案")]
                )
            )
            return

        try:
            # 下載檔案
            file_path = download_line_file(event.message.id, file_name)
            print(f"檔案已下載：{file_path}")

            # 上傳到 Gemini
            uploaded_file = upload_file_to_gemini(file_path)
            print(f"檔案已上傳到 Gemini：{uploaded_file.uri}")

            # 儲存到使用者 session
            session = get_user_session(user_id)
            session["uploaded_file"] = uploaded_file

            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=f"✅ 檔案「{file_name}」上傳成功！\n\n現在您可以輸入「AI 問題」來詢問關於這份文件的問題。\n\n輸入「清除文件」可恢復一般對話模式。")]
                )
            )
        except Exception as e:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=f"❌ 檔案處理失敗：{str(e)}")]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:__main__:Request body: {"destination":"Uc0bc2480f5fd2dd98684ea589fdf9de6","events":[]}
INFO:werkzeug:127.0.0.1 - - [02/Jan/2026 12:02:31] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"Uc0bc2480f5fd2dd98684ea589fdf9de6","events":[]}


INFO:__main__:Request body: {"destination":"Uc0bc2480f5fd2dd98684ea589fdf9de6","events":[{"type":"message","message":{"type":"file","id":"594792671470945005","markAsReadToken":"31Z6dcx4SORtUehZvmAHIuVFhYZS4mm9E2l-9o9f0bWVxz85rkqu59QdhUwrjK_fOh69Yv2RsVo_ws564_pO2UriaxlIGM1TqcNMRHNaRh2rni-s-T9ItLLlY1bCaCcI8wNb5JJVWIezT3prsTHeMQ-eIKmhFn6hFavZ5HCDma3Pbi6NKC_m-qG3sRqxrK9CSfYugCVUiP79IRISkK6TJQ","fileName":"faq.txt.txt","fileSize":30,"contentProvider":{"type":"line"}},"webhookEventId":"01KDZ9DC2JYXF3CR2215X9ZHGW","deliveryContext":{"isRedelivery":false},"timestamp":1767355363052,"source":{"type":"user","userId":"Uafbaa1ab617f08637f77d4fdd09c7975"},"replyToken":"4f57c1a206c0415f8bee5c782eff6d9e","mode":"active"}]}


BODY:  {"destination":"Uc0bc2480f5fd2dd98684ea589fdf9de6","events":[{"type":"message","message":{"type":"file","id":"594792671470945005","markAsReadToken":"31Z6dcx4SORtUehZvmAHIuVFhYZS4mm9E2l-9o9f0bWVxz85rkqu59QdhUwrjK_fOh69Yv2RsVo_ws564_pO2UriaxlIGM1TqcNMRHNaRh2rni-s-T9ItLLlY1bCaCcI8wNb5JJVWIezT3prsTHeMQ-eIKmhFn6hFavZ5HCDma3Pbi6NKC_m-qG3sRqxrK9CSfYugCVUiP79IRISkK6TJQ","fileName":"faq.txt.txt","fileSize":30,"contentProvider":{"type":"line"}},"webhookEventId":"01KDZ9DC2JYXF3CR2215X9ZHGW","deliveryContext":{"isRedelivery":false},"timestamp":1767355363052,"source":{"type":"user","userId":"Uafbaa1ab617f08637f77d4fdd09c7975"},"replyToken":"4f57c1a206c0415f8bee5c782eff6d9e","mode":"active"}]}
檔案已下載：/content/uploaded_files/faq.txt.txt
檔案已上傳到 Gemini：https://generativelanguage.googleapis.com/v1beta/files/kh0fzjaa5z8r


INFO:werkzeug:127.0.0.1 - - [02/Jan/2026 12:02:45] "POST / HTTP/1.1" 200 -
INFO:__main__:Request body: {"destination":"Uc0bc2480f5fd2dd98684ea589fdf9de6","events":[{"type":"message","message":{"type":"text","id":"594792712792965411","quoteToken":"2dDEwRnb7I7GHgiYWtd4rT-P5pX3XgsQqSfGd8f58AZRsKa756eIEP4UnORb7fBt-qpbpfSQud53Q7XcHYznMriz_UkfGfIbnnEoGjRJfnqXUAJlw5qkW8aWvVmWVMSZb8R8VR1hBY3CqkPiCTP-5A","markAsReadToken":"aPhuQ6XRpcu3f6YM1oHN4mZS4fbgbrF7oa5LgiWu6saAOBwTdYEvNQgHCrX9lqahDv3dkbZwSTirpjXyMiiO6ETfQCxUrHzMIuk1yL21-3f2bBq0zKeYT2-sQxuyKUzlvlS8dbJZIY8L-J-coe-fe4IxiDT-iVZBjhCiFJLVZ-pTt6CUDmpPwb8ygtLpb5cux85HItyvMr5Zngmi2nlOjQ","text":"AI 奧斯卡家的貓叫甚麼"},"webhookEventId":"01KDZ9E4AZQTQQZ1MM6VN9QNDZ","deliveryContext":{"isRedelivery":false},"timestamp":1767355387788,"source":{"type":"user","userId":"Uafbaa1ab617f08637f77d4fdd09c7975"},"replyToken":"41d139908e964b44aa4d83449d27cdcb","mode":"active"}]}


BODY:  {"destination":"Uc0bc2480f5fd2dd98684ea589fdf9de6","events":[{"type":"message","message":{"type":"text","id":"594792712792965411","quoteToken":"2dDEwRnb7I7GHgiYWtd4rT-P5pX3XgsQqSfGd8f58AZRsKa756eIEP4UnORb7fBt-qpbpfSQud53Q7XcHYznMriz_UkfGfIbnnEoGjRJfnqXUAJlw5qkW8aWvVmWVMSZb8R8VR1hBY3CqkPiCTP-5A","markAsReadToken":"aPhuQ6XRpcu3f6YM1oHN4mZS4fbgbrF7oa5LgiWu6saAOBwTdYEvNQgHCrX9lqahDv3dkbZwSTirpjXyMiiO6ETfQCxUrHzMIuk1yL21-3f2bBq0zKeYT2-sQxuyKUzlvlS8dbJZIY8L-J-coe-fe4IxiDT-iVZBjhCiFJLVZ-pTt6CUDmpPwb8ygtLpb5cux85HItyvMr5Zngmi2nlOjQ","text":"AI 奧斯卡家的貓叫甚麼"},"webhookEventId":"01KDZ9E4AZQTQQZ1MM6VN9QNDZ","deliveryContext":{"isRedelivery":false},"timestamp":1767355387788,"source":{"type":"user","userId":"Uafbaa1ab617f08637f77d4fdd09c7975"},"replyToken":"41d139908e964b44aa4d83449d27cdcb","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [02/Jan/2026 12:03:10] "POST / HTTP/1.1" 200 -
